In [7]:
from sqlalchemy import create_engine
from sqlalchemy import text
from db_auth import getConnectionInfo

login = getConnectionInfo()

username=login['username']
password=login['password']
host=login['host']
port=login['port']
db_name=login['db_name']

engine = create_engine('postgresql://'+username+':'+password+'@'+host+':'+port+'/'+db_name, echo=True, future=True)

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())
    
    conn.execute(text("CREATE TABLE FUNC( x integer, y integer )"))
    conn.execute(text("INSERT INTO FUNC (x,y) VALUES (:x, :y)"),
                 [{"x":1, "y":1}, {"x" : 2, "y":4}])
    conn.commit()
    


2021-11-17 14:48:23,202 INFO sqlalchemy.engine.Engine select version()
2021-11-17 14:48:23,202 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-11-17 14:48:23,203 INFO sqlalchemy.engine.Engine select current_schema()
2021-11-17 14:48:23,204 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-11-17 14:48:23,205 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2021-11-17 14:48:23,205 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-11-17 14:48:23,207 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-17 14:48:23,207 INFO sqlalchemy.engine.Engine select 'hello world'
2021-11-17 14:48:23,208 INFO sqlalchemy.engine.Engine [generated in 0.00102s] {}
[('hello world',)]
2021-11-17 14:48:23,209 INFO sqlalchemy.engine.Engine CREATE TABLE FUNC( x integer, y integer )
2021-11-17 14:48:23,210 INFO sqlalchemy.engine.Engine [generated in 0.00050s] {}
2021-11-17 14:48:23,214 INFO sqlalchemy.engine.Engine INSERT INTO FUNC (x,y) VALUES (%(x)s, %(y)s)
2021-11-17 14:48:23,215 INFO sqlalchemy.e

In [8]:
with engine.begin() as conn:
    conn.execute(text("INSERT INTO FUNC (x,y) VALUES (:x, :y)"),
                 [{"x":6, "y":8}, {"x" : 9, "y":10}])

2021-11-17 14:48:30,694 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-17 14:48:30,695 INFO sqlalchemy.engine.Engine INSERT INTO FUNC (x,y) VALUES (%(x)s, %(y)s)
2021-11-17 14:48:30,696 INFO sqlalchemy.engine.Engine [cached since 7.482s ago] ({'x': 6, 'y': 8}, {'x': 9, 'y': 10})
2021-11-17 14:48:30,699 INFO sqlalchemy.engine.Engine COMMIT


In [38]:
#statement Execution
with engine.connect() as conn:
    result = conn.execute(text("SELECT x,y FROM FUNC;"))
    for row in result:
        print(f"x: {row.x} y:{row.y}")
        
    result = conn.execute(text("SELECT x,y FROM FUNC;"))
    for dict_row in result.mappings():
        x = dict_row['x']
        y = dict_row['y']
        print(x,y)
    
    result=conn.execute(text("SELECT x,y FROM FUNC where y > :y"),
                        {"y":2})
    for row in result:
        print(f"x: {row.x} y:{row.y}")
        
    conn.execute(text("INSERT INTO FUNC(x,y) VALUES (:x, :y)"),
                 [{"x":11, "y":12}, {"x":13, "y": 14}]) #insert문 반환값 없음
    conn.commit()

2021-11-17 15:53:42,156 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-17 15:53:42,157 INFO sqlalchemy.engine.Engine SELECT x,y FROM FUNC;
2021-11-17 15:53:42,158 INFO sqlalchemy.engine.Engine [cached since 2049s ago] {}
x: 1 y:1
x: 2 y:4
x: 6 y:8
x: 9 y:10
2021-11-17 15:53:42,159 INFO sqlalchemy.engine.Engine SELECT x,y FROM FUNC;
2021-11-17 15:53:42,160 INFO sqlalchemy.engine.Engine [cached since 2049s ago] {}
1 1
2 4
6 8
9 10
2021-11-17 15:53:42,161 INFO sqlalchemy.engine.Engine SELECT x,y FROM FUNC where y > %(y)s
2021-11-17 15:53:42,162 INFO sqlalchemy.engine.Engine [cached since 619.2s ago] {'y': 2}
x: 2 y:4
x: 6 y:8
x: 9 y:10
2021-11-17 15:53:42,164 INFO sqlalchemy.engine.Engine INSERT INTO FUNC(x,y) VALUES (%(x)s, %(y)s)
2021-11-17 15:53:42,164 INFO sqlalchemy.engine.Engine [generated in 0.00053s] ({'x': 11, 'y': 12}, {'x': 13, 'y': 14})
2021-11-17 15:53:42,165 INFO sqlalchemy.engine.Engine COMMIT


In [41]:
#bundling parameters
with engine.connect() as conn:
    stmt = text("SELECT x, y FROM FUNC WHERE y > :y ORDER BY x,y").bindparams(y=6)
    result = conn.execute(stmt)
    for row in result:
        print(f"x: {row.x} y:{row.y}")

2021-11-17 16:02:09,285 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-17 16:02:09,286 INFO sqlalchemy.engine.Engine SELECT x, y FROM FUNC WHERE y > %(y)s ORDER BY x,y
2021-11-17 16:02:09,287 INFO sqlalchemy.engine.Engine [cached since 130.5s ago] {'y': 6}
x: 6 y:8
x: 9 y:10
x: 11 y:12
x: 13 y:14
2021-11-17 16:02:09,288 INFO sqlalchemy.engine.Engine ROLLBACK


In [44]:
#ORM Session으로 실행하기
from sqlalchemy.orm import Session
stmt = text("SELECT x, y FROM FUNC WHERE y > :y ORDER BY x,y").bindparams(y=6)
with Session(engine) as session:
    result = session.execute(stmt)
    for row in result:
        print(f"x: {row.x} y:{row.y}")
    
    result = session.execute(
        text("UPDATE FUNC SET y=:y WHERE x=:x"),
        [{"x":9, "y":11}, {"x":13, "y":15}]
    )
    session.commit() #커밋해야 저장됨

2021-11-17 16:11:04,735 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-17 16:11:04,737 INFO sqlalchemy.engine.Engine SELECT x, y FROM FUNC WHERE y > %(y)s ORDER BY x,y
2021-11-17 16:11:04,737 INFO sqlalchemy.engine.Engine [cached since 666s ago] {'y': 6}
x: 6 y:8
x: 9 y:10
x: 11 y:12
x: 13 y:14
2021-11-17 16:11:04,739 INFO sqlalchemy.engine.Engine UPDATE FUNC SET y=%(y)s WHERE x=%(x)s
2021-11-17 16:11:04,739 INFO sqlalchemy.engine.Engine [cached since 25.81s ago] ({'y': 11, 'x': 9}, {'y': 15, 'x': 13})
2021-11-17 16:11:04,741 INFO sqlalchemy.engine.Engine COMMIT
